# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770349741538                   -0.53    9.0    169ms
  2   -2.771683491446       -2.87       -1.31    1.0   89.6ms
  3   -2.771714105943       -4.51       -2.58    1.0   89.6ms
  4   -2.771714677391       -6.24       -3.84    1.0   92.1ms
  5   -2.771714714688       -7.43       -4.14    2.0    113ms
  6   -2.771714715219       -9.27       -5.25    1.0   96.4ms
  7   -2.771714715246      -10.57       -5.28    2.0    123ms
  8   -2.771714715249      -11.44       -6.18    1.0    101ms
  9   -2.771714715250      -12.74       -6.66    2.0    119ms
 10   -2.771714715250   +  -13.82       -7.37    1.0    117ms
 11   -2.771714715250      -13.77       -8.03    1.0    116ms


-0.00013457245463585415

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770511938599                   -0.52    9.0    201ms
  2   -2.771779371755       -2.90       -1.33    1.0   90.5ms
  3   -2.771801523886       -4.65       -2.42    1.0    101ms
  4   -2.771801995822       -6.33       -3.12    1.0    103ms
  5   -2.771802074213       -7.11       -4.51    2.0    112ms
  6   -2.771802074405       -9.72       -4.77    1.0    102ms
  7   -2.771802074471      -10.18       -5.29    2.0    118ms
  8   -2.771802074473      -11.69       -5.44    1.0    102ms
  9   -2.771802074476      -11.54       -6.41    1.0    107ms
 10   -2.771802074476      -13.36       -7.21    2.0    142ms
 11   -2.771802074476   +  -14.45       -7.36    1.0    103ms
 12   -2.771802074476      -14.35       -8.87    1.0    108ms


0.017612221374873414

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457245463585415
Displaced dipole:  0.017612221374873414
Polarizability :   1.7746793829509266


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920856772e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551379403e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852762182886e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694602608886e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088786663825e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.277856363214e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 8.254207196877e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 8.254207196877e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 6.633846995251e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.062542518768e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 2.269117845075e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 2.269117845075e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 2.2692549922413475e-13
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.